Objectives:

-Clean up dataset

-Find data errors

-If possible correct data errors

Data:
    - Various levels of granularity one row per loan, one loan per accounting/cash flow event
    
A calendar monthly summary of a loan's life cycle from
origination thru monthly payments to
maturity/default/prepayment.

These are monthly-paying amortizing loans
that can be **fully or partially prepaid at any time without
penalty.**

Loans frequently go delinquent and if any payment exceeds ~120 days late (dpd>120) that loan defaults.


- brief summary of problem and approachs/solution

- data quality report detailing frequency of errors

- documented code (and executable if relevant)

- cleaned dataset


** Data Dictionary **

- loanId : A unique identifier assigned to each loan
- mob : 'month on balance'. An integer indexing calendar months for each
loan starting at zero at loan origination. The first scheduled payment
falls in month on balance one.
- loanAmount : Amount of loan in US$
- origDate : Date of loan origination. Funds are disbursed on this
date.
- obsDate : Observation Date. Loan status, specifically the 'dpd'
value is measured as of the end of this day, the last day of the calendar month.
- dueDate : The monthly scheduled payment due date. This is fixed at
origination date. Each month an additional payment is due. When
payments are received they are applied to the earliest due payment.
- dpd : 'Days Past Due', measures the worst status achieved for a loan
within the calendar month in terms of days late. 'dpd' is a monthly
maximum of a daily series you don't have access to. The daily days
past due variable counts the number of days the oldest due payment is late.
- dpdEom : 'Days Past Due End Of Month'. This the value of the daily
days past due on the observation date. The number of days late for the
oldest unsatisfied payment. For instance if a payment is late
intra-month but made by the end of the month, then dpd will be
non-zero and dpdEom will be zero.


In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv('/Users/armanarkilic/Desktop/BlueElephant/becm_challenge_data.csv')

In [3]:
raw_data.head()

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
0,40258,0,4000,2010-01-04,2010-01-31,NaN,0,0
1,40258,1,4000,2010-01-04,2010-02-28,2010-02-04,0,0
2,40258,2,4000,2010-01-04,2010-03-31,2010-03-04,0,0
3,40258,3,4000,2010-01-04,2010-04-30,2010-04-04,0,0
4,40258,4,4000,2010-01-04,2010-05-31,2010-05-04,0,0


In [4]:
raw_data.tail()

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
12491640,813384,1,5500,2017-05-31,2017-06-30,2017-06-30,0,0
12491641,813387,0,2000,2017-05-31,2017-05-31,NaN,0,0
12491642,813387,1,2000,2017-05-31,2017-06-30,2017-06-30,0,0
12491643,813390,0,11000,2017-05-31,2017-05-31,NaN,0,0
12491644,813390,1,11000,2017-05-31,2017-06-30,2017-06-30,0,0


In [5]:
# At first glance, I see that there are plenty of NaN for due date column

In [6]:
# Check which column includes NaN
raw_data.isnull().any()


loanId        False
mob           False
loanAmount    False
origDate      False
obsDate       False
dueDate        True
dpd           False
dpdEom        False
dtype: bool

In [7]:
# Good news, NaN isolated to due date, let's find how many NaN in this column
nan_ct = sum(pd.isnull(raw_data['dueDate']))

# What percentage of the dueDate is problematic?
print(nan_ct/raw_data.shape[0]*100)

# So about 5% of the dueDates are missing. Is there a way to recover this info?

5.59245799893


In [8]:
# I could simply drop this information: raw_data.dropna() but I think there is a nicer solution
# I see that there are many similar loanId, some have dueDate information some don't. I will get the
# indexes of those missing the dueDate and extract the loanIds. Then fetch the dueDate using these loanIds.

nan_index = pd.isnull(raw_data).any(1).nonzero()[0]
print(nan_index)
print(len(nan_index))
assert nan_ct == len(nan_index)

[       0       37       70 ..., 12491639 12491641 12491643]
698590


In [9]:
# Assert does not raise and visual inspection suggests I got the right index.
# Lets capture the very first element in it and show you guys this is one of the missing rows
raw_data.ix[nan_index[0]]

loanId             40258
mob                    0
loanAmount          4000
origDate      2010-01-04
obsDate       2010-01-31
dueDate              NaN
dpd                    0
dpdEom                 0
Name: 0, dtype: object

In [10]:
# Now compose a list of all loanIds with NaN dueDate

nan_loanIds = [raw_data.ix[i]['loanId'] for i in nan_index]

In [11]:
# Visually confirm this returns something that makes sense, it does as the info from head and tail shows
print(nan_loanIds[0])
print(nan_loanIds[-1])

40258
813390


In [12]:
# Next, I want to verify if I am on the right track. The best way to do this is to visually inspect a single loan.
# For no particular reason, I pick the first loan to inspect. 
raw_data.loc[raw_data['loanId']==nan_loanIds[0]]

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
0,40258,0,4000,2010-01-04,2010-01-31,NaN,0,0
1,40258,1,4000,2010-01-04,2010-02-28,2010-02-04,0,0
2,40258,2,4000,2010-01-04,2010-03-31,2010-03-04,0,0
3,40258,3,4000,2010-01-04,2010-04-30,2010-04-04,0,0
4,40258,4,4000,2010-01-04,2010-05-31,2010-05-04,0,0
5,40258,5,4000,2010-01-04,2010-06-30,2010-06-04,0,0
6,40258,6,4000,2010-01-04,2010-07-31,2010-07-04,0,0
7,40258,7,4000,2010-01-04,2010-08-31,2010-08-04,0,0
8,40258,8,4000,2010-01-04,2010-09-30,2010-09-04,0,0
9,40258,9,4000,2010-01-04,2010-10-31,2010-10-04,0,0


In [13]:
# I see that my initial assumption that having NaN in the due date as "missing" data is not very accurate.
# In case of loan 40258, the only missing due date is the very first month, the day loan was originated, which makes
# sense. If issued on the 4th of January, this would be due on the 2nd of February.

In [14]:
# I want to verify that I am correct with my above finding. I will pick a random number
# and investigate it like I did above.

In [15]:
# Pick one index from a Gaussian
rix = int(abs(np.ceil(np.random.normal()*10000)))
raw_data.loc[raw_data['loanId']==nan_loanIds[rix]]

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
166673,47715,0,2500,2011-03-14,2011-03-31,NaN,0,0
166674,47715,1,2500,2011-03-14,2011-04-30,2011-04-14,0,0
166675,47715,2,2500,2011-03-14,2011-05-31,2011-05-14,0,0
166676,47715,3,2500,2011-03-14,2011-06-30,2011-06-14,0,0
166677,47715,4,2500,2011-03-14,2011-07-31,2011-07-14,0,0
166678,47715,5,2500,2011-03-14,2011-08-31,2011-08-14,0,0
166679,47715,6,2500,2011-03-14,2011-09-30,2011-09-14,0,0


In [16]:
# The above snippet shows that the point I made earlier is on point.
# I created about ~10 random numbers and got a similar result. 

In [17]:
# So far I did quite a bit of work but my earlier assumption of dueDate NaN being problematic was not accurate.
# It is not all that bad, this allowed me to explore the data and understand how each individual loan is structured. 

In [18]:
# I think I will be able to understand the data better and spot problems easier if I focus on individual loans.
# The best way to do this is the sort the data frame on loanId AND mob as mob sequentially increases

In [19]:
sorted_data = raw_data.sort_values(by=['loanId', 'mob'])
sorted_data.head()

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
0,40258,0,4000,2010-01-04,2010-01-31,NaN,0,0
1,40258,1,4000,2010-01-04,2010-02-28,2010-02-04,0,0
2,40258,2,4000,2010-01-04,2010-03-31,2010-03-04,0,0
3,40258,3,4000,2010-01-04,2010-04-30,2010-04-04,0,0
4,40258,4,4000,2010-01-04,2010-05-31,2010-05-04,0,0


In [20]:
sorted_data.tail()


,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
12491640,813384,1,5500,2017-05-31,2017-06-30,2017-06-30,0,0
12491641,813387,0,2000,2017-05-31,2017-05-31,NaN,0,0
12491642,813387,1,2000,2017-05-31,2017-06-30,2017-06-30,0,0
12491643,813390,0,11000,2017-05-31,2017-05-31,NaN,0,0
12491644,813390,1,11000,2017-05-31,2017-06-30,2017-06-30,0,0


In [21]:
# Once I sorted the data and verified it's been sorted properly, I started to explore it further.
# What I mean by further is basically paginate using the indexes and look for abnormalities.
# As I was going along, I noticed a potential issue with the 'dpd'

sorted_data.iloc[50:80]

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
50,40261,13,1000,2010-01-04,2011-02-28,2011-02-04,0,0
51,40261,14,1000,2010-01-04,2011-03-31,2011-03-04,0,0
52,40261,15,1000,2010-01-04,2011-04-30,2011-04-04,0,0
53,40261,16,1000,2010-01-04,2011-05-31,2011-05-04,0,0
54,40261,17,1000,2010-01-04,2011-06-30,2011-06-04,0,0
55,40261,18,1000,2010-01-04,2011-07-31,2011-07-04,0,0
56,40261,19,1000,2010-01-04,2011-08-31,2011-08-04,0,0
57,40261,20,1000,2010-01-04,2011-09-30,2011-09-04,0,0
58,40261,21,1000,2010-01-04,2011-10-31,2011-10-04,0,0
59,40261,22,1000,2010-01-04,2011-11-30,2011-11-04,11,0


In [22]:
sorted_data.iloc[475:500]

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
475,40291,0,1500,2010-01-06,2010-01-31,NaN,0,0
476,40291,1,1500,2010-01-06,2010-02-28,2010-02-06,0,0
477,40291,2,1500,2010-01-06,2010-03-31,2010-03-06,0,0
478,40291,3,1500,2010-01-06,2010-04-30,2010-04-06,0,0
479,40291,4,1500,2010-01-06,2010-05-31,2010-05-06,0,0
480,40291,5,1500,2010-01-06,2010-06-30,2010-06-06,0,0
481,40291,6,1500,2010-01-06,2010-07-31,2010-07-06,0,0
482,40291,7,1500,2010-01-06,2010-08-31,2010-08-06,0,0
483,40291,8,1500,2010-01-06,2010-09-30,2010-09-06,0,0
484,40291,9,1500,2010-01-06,2010-10-31,2010-10-06,7,0


In [23]:
# Lets focus on loanId=40264, one of the loans I think is problematic
loan40264 = sorted_data[sorted_data['loanId']==40264]

In [24]:
loan40264

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
70,40264,0,1000,2010-01-04,2010-01-31,NaN,0,0
71,40264,1,1000,2010-01-04,2010-02-28,2010-02-04,0,0
72,40264,2,1000,2010-01-04,2010-03-31,2010-03-04,0,0
73,40264,3,1000,2010-01-04,2010-04-30,2010-04-04,0,0
74,40264,4,1000,2010-01-04,2010-05-31,2010-05-04,23,0
75,40264,5,1000,2010-01-04,2010-06-30,2010-06-04,57,57
76,40264,6,1000,2010-01-04,2010-07-31,2010-07-04,88,88
77,40264,7,1000,2010-01-04,2010-08-31,2010-08-04,119,119
78,40264,8,1000,2010-01-04,2010-09-30,2010-09-04,121,0


 In the README.txt, dpd is defined as:
 
 dpd : 'Days Past Due', measures the worst status achieved for a loan
**within the calendar month** in terms of days late. 'dpd' is a monthly
maximum of a daily series you don't have access to. The daily days
past due variable counts the number of days the oldest due payment is late.


For instance, from 2010-05-31 to 2010-06-30, 30 days have elapsed and no payments have been made towards the loan.When the loan is observed at 2010-06-30, the dpd value is at 57 compared to 23 30 days earlier. 

If I look at 2010-06-30	(dpd: 57) and 2010-07-31(dpd: 88), I observe that in this interval no payment is done like the earlier case and 31 days have passed. 88-57=31 which is equal to the number of days elapsed, which tells me there is something wrong with the increments with the dpd reporting on 2010-06-30.

In [25]:
# I need to calculate dpd properly, how do I do that? I will figure out on this small data frame
# below and apply it globally once I show it works:

loan40264 = sorted_data[sorted_data['loanId']==40264]
# First of all, etc.:

# 1.Convert all dates that are str to datetime object to do subtractions 
loan40264['obsDate' ] =  pd.to_datetime(loan40264['obsDate'])

# 2. Ensure the dpd diff does not exceede the number of days
# Calculate rolling number of days 

mob_size = loan40264.shape[0]
for i in range(mob_size-1):
    dpd0 = loan40264[loan40264['mob']==i]['dpd']
    dpd1 = loan40264[loan40264['mob']==i+1]['dpd']
    x = dpd1.values - dpd0.values
    dpd_diff = x[0]
    ts = loan40264[loan40264['mob']==i+1].iloc[0]['obsDate'] - loan40264[loan40264['mob']==i].iloc[0]['obsDate']
    days_diff = ts.days
    eom = loan40264[loan40264['mob']==i+1]['dpdEom'].values[0]
    
    #  Make sure the debt is not paid dpdEom !=0 and the diff issue explained above exists
    if days_diff < dpd_diff and eom != 0:
        actual =  loan40264[loan40264['mob']==i]['dpd'].values[0] + days_diff
        loan40264 = loan40264.replace(dpd1.values[0], actual)
loan40264

/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
70,40264,0,1000,2010-01-04,2010-01-31,NaN,0,0
71,40264,1,1000,2010-01-04,2010-02-28,2010-02-04,0,0
72,40264,2,1000,2010-01-04,2010-03-31,2010-03-04,0,0
73,40264,3,1000,2010-01-04,2010-04-30,2010-04-04,0,0
74,40264,4,1000,2010-01-04,2010-05-31,2010-05-04,23,0
75,40264,5,1000,2010-01-04,2010-06-30,2010-06-04,53,53
76,40264,6,1000,2010-01-04,2010-07-31,2010-07-04,84,84
77,40264,7,1000,2010-01-04,2010-08-31,2010-08-04,115,115
78,40264,8,1000,2010-01-04,2010-09-30,2010-09-04,121,0


In [26]:
# As you can see above, the code snippet works and modified the values properly, fixing the time issue.Updating both dpd
# and dpdEom appropriately.
# However, this only works for this specific data frame I created using local data. 
# I need to generalize it to all 

# PS: The warning is due to pandas API changes. I am aware of this issue and how to fix it but I think at this point
# I decided not to spend time on it. I am aware warnings eventually turn into errors and would definitely address it
# if time permitted.

In [27]:
# From the local dataframe operations above, I created a function to apply globally
def fix_dpd(df):
    # First of all, etc.:

    # 1.Convert all dates that are str to datetime object to do subtractions 
    df['obsDate' ] =  pd.to_datetime(df['obsDate'])

    # 2. Ensure the dpd diff does not exceede the number of days
    # Calculate rolling number of days 
    issue_ctr = 0

    mob_size = df.shape[0]
    for i in range(mob_size-1):
        dpd0 = df[df['mob']==i]['dpd']
        dpd1 = df[df['mob']==i+1]['dpd']
        x = dpd1.values - dpd0.values
        dpd_diff = x[0]
        ts = df[df['mob']==i+1].iloc[0]['obsDate'] - df[df['mob']==i].iloc[0]['obsDate']
        days_diff = ts.days
        eom = df[df['mob']==i+1]['dpdEom'].values[0]

        #  Make sure the debt is not paid dpdEom !=0 and the diff issue explained above exists
        if days_diff < dpd_diff and eom != 0:
            actual =  df[df['mob']==i]['dpd'].values[0] + days_diff
            df = df.replace(dpd1.values[0], actual)
            issue_ctr += 1 # to collect statistics
    return issue_ctr
        


In [28]:
# Get all non-zero dpds and find the loanIds corresponding to these
# I do this using a simple one liner that allows me to isolate loans with non-zero dpd and use uniqueness of the
# loanIds to extract what I need.

non_zero_dbds = sorted_data[sorted_data['dpd']!=0].loanId.unique()
non_zero_dbds

array([ 40261,  40264,  40267, ..., 806484, 806601, 807039])

In [29]:
# Lets verify the indexes I extracted above are accurate visually!

df = sorted_data[sorted_data['loanId']==non_zero_dbds[40]]
df

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
3272,40415,0,3000,2010-01-19,2010-01-31,NaN,0,0
3273,40415,1,3000,2010-01-19,2010-02-28,2010-02-19,0,0
3274,40415,2,3000,2010-01-19,2010-03-31,2010-03-19,0,0
3275,40415,3,3000,2010-01-19,2010-04-30,2010-04-19,0,0
3276,40415,4,3000,2010-01-19,2010-05-31,2010-05-19,0,0
3277,40415,5,3000,2010-01-19,2010-06-30,2010-06-19,0,0
3278,40415,6,3000,2010-01-19,2010-07-31,2010-07-19,0,0
3279,40415,7,3000,2010-01-19,2010-08-31,2010-08-19,0,0
3280,40415,8,3000,2010-01-19,2010-09-30,2010-09-19,0,0
3281,40415,9,3000,2010-01-19,2010-10-31,2010-10-19,0,0


In [30]:
# Now, let's find out whether the function I defined works on these smaller data frames

fix_dpd(df)
df

/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
3272,40415,0,3000,2010-01-19,2010-01-31,NaN,0,0
3273,40415,1,3000,2010-01-19,2010-02-28,2010-02-19,0,0
3274,40415,2,3000,2010-01-19,2010-03-31,2010-03-19,0,0
3275,40415,3,3000,2010-01-19,2010-04-30,2010-04-19,0,0
3276,40415,4,3000,2010-01-19,2010-05-31,2010-05-19,0,0
3277,40415,5,3000,2010-01-19,2010-06-30,2010-06-19,0,0
3278,40415,6,3000,2010-01-19,2010-07-31,2010-07-19,0,0
3279,40415,7,3000,2010-01-19,2010-08-31,2010-08-19,0,0
3280,40415,8,3000,2010-01-19,2010-09-30,2010-09-19,0,0
3281,40415,9,3000,2010-01-19,2010-10-31,2010-10-19,0,0


 As we can see above, the function works properly! It handles corner cases that there is no issue with data or data is simply zero

In [31]:
# Now that we know this function works, I can apply it to the entire dataframe. Below, I am showing the basic idea
# that I am using. if you recall above, I am using small data frames to figure out the problems with dpd. In order to
# do this, I am decomposing my data frame into smaller data frames. Below is a demo of the technique I use to do this.
# I like showing it with small data frames so that it will make sense to you later as I tackle as a whole.


df1 = sorted_data[sorted_data['loanId']==non_zero_dbds[40]]
df2 = sorted_data[sorted_data['loanId']==non_zero_dbds[41]]
fix_dpd(df1)
fix_dpd(df2)
frames = [df1, df2]
result = pd.concat(frames)

/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [32]:
result

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
3272,40415,0,3000,2010-01-19,2010-01-31,NaN,0,0
3273,40415,1,3000,2010-01-19,2010-02-28,2010-02-19,0,0
3274,40415,2,3000,2010-01-19,2010-03-31,2010-03-19,0,0
3275,40415,3,3000,2010-01-19,2010-04-30,2010-04-19,0,0
3276,40415,4,3000,2010-01-19,2010-05-31,2010-05-19,0,0
3277,40415,5,3000,2010-01-19,2010-06-30,2010-06-19,0,0
3278,40415,6,3000,2010-01-19,2010-07-31,2010-07-19,0,0
3279,40415,7,3000,2010-01-19,2010-08-31,2010-08-19,0,0
3280,40415,8,3000,2010-01-19,2010-09-30,2010-09-19,0,0
3281,40415,9,3000,2010-01-19,2010-10-31,2010-10-19,0,0


In [33]:
# This captures all of the rows with dpd=0
df4 = sorted_data[sorted_data['dpd']==0] 

In [35]:
dfs = []
dfs.append(df4)
for i in non_zero_dbds:
    df = sorted_data[sorted_data['loanId']==non_zero_dbds[i]]
    fix_dpd(df)
    dfs.append(df)
# This method is very slow but works. I would do it differently if time permitted!

/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
cleaned_up_data = pd.concat(dfs)

In [38]:
cleaned_up_data.head()

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
0,40258,0,4000,2010-01-04,2010-01-31,NaN,0,0
1,40258,1,4000,2010-01-04,2010-02-28,2010-02-04,0,0
2,40258,2,4000,2010-01-04,2010-03-31,2010-03-04,0,0
3,40258,3,4000,2010-01-04,2010-04-30,2010-04-04,0,0
4,40258,4,4000,2010-01-04,2010-05-31,2010-05-04,0,0


In [39]:
cleaned_up_data.shape

(12491645, 8)

In [40]:
# Next, I check whether there are any loans in the data that are deragatory dpd >120 and whether there are any loans 
# with dpd/dpdEom < 5 as these loans should also have a zero dpdEom



# First check if any of the loans have dpd > 120:
cleaned_up_data[cleaned_up_data['dpd']>120]

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
78,40264,8,1000,2010-01-04,2010-09-30,2010-09-04,121,0
414,40288,35,1200,2010-01-06,2012-12-31,2012-12-06,121,0
566,40294,26,1000,2010-01-06,2012-03-31,2012-03-06,121,0
605,40296,33,3000,2010-01-06,2012-10-31,2012-10-06,121,0
802,40304,15,4000,2010-01-06,2011-04-30,2011-04-06,121,0
1065,40313,7,3500,2010-01-07,2010-08-31,2010-08-07,121,0
1138,40316,37,2200,2010-01-07,2013-02-28,2013-02-07,121,0
1460,40330,15,7000,2010-01-08,2011-04-30,2011-04-08,121,0
1565,40336,15,10000,2010-01-11,2011-04-30,2011-04-11,121,0
1608,40340,36,2500,2010-01-08,2013-01-31,2013-01-08,121,0


In [41]:
# As we can see above 9071 of these loans have dbd > 120! We know these loans should be defaulted.
pct_of_dbd_120 = cleaned_up_data[cleaned_up_data['dpd']>120].shape[0] / cleaned_up_data.shape[0] * 100
print('% of the loans that are unpaid more than 120 days and should expire', pct_of_dbd_120)

% of the loans that are unpaid more than 120 days and should expire 0.07261653689325945


In [42]:
# Drop these loans that already deragatory!
cleaned_up_data = cleaned_up_data.drop(cleaned_up_data['dpd']>120)

In [43]:
# Now, check the ratio dpd/dpdEom
dpd_positives = cleaned_up_data[cleaned_up_data['dpd']!=0]


In [44]:
ratios = dpd_positives.dpd/dpd_positives.dpdEom
ratios = ratios.replace(np.inf, 0)
np.sum(ratios>=5)

2213

In [45]:
# We observe that dpd/dpdEom ratios that are 5 or more are not reported as zero. The percentage of this is:
np.sum(ratios>=5)/cleaned_up_data.shape[0] * 100

0.01771584410473466

In [46]:
# We deal with this by setting all of these values to zero as they should've been done in the first place
cleaned_up_data[cleaned_up_data['dpd']/cleaned_up_data['dpdEom']>=5]

,loanId,mob,loanAmount,origDate,obsDate,dueDate,dpd,dpdEom
59,40261,22,1000,2010-01-04,2011-11-30,2011-11-04,11,0
65,40261,28,1000,2010-01-04,2012-05-31,2012-05-04,11,0
74,40264,4,1000,2010-01-04,2010-05-31,2010-05-04,23,0
78,40264,8,1000,2010-01-04,2010-09-30,2010-09-04,121,0
84,40267,5,1500,2010-01-04,2010-06-30,2010-06-04,9,0
94,40267,15,1500,2010-01-04,2011-04-30,2011-04-04,17,0
414,40288,35,1200,2010-01-06,2012-12-31,2012-12-06,121,0
484,40291,9,1500,2010-01-06,2010-10-31,2010-10-06,7,0
485,40291,10,1500,2010-01-06,2010-11-30,2010-11-06,23,0
489,40291,14,1500,2010-01-06,2011-03-31,2011-03-06,119,0


In [47]:
cleaned_up_data.to_csv('example.csv')